# RAG 应用评估

RAG 应用评估是一个复杂的问题，完整的评估有很多指标。

## 指标选择

开源框架 ragas 有如下指标。

| 阶段 | 指标名称 | 指标描述 | 评估方法 | 是否需要参考答案(`ground truth`) |
| --- | --- | --- | --- | --- | 
| 检索 | 上下文召回 Context Recall | 检索到的上下文`context`与参考答案`ground truth`的一致性程度 | LLM | 是 |
| 检索 | 上下文精确度 Context Precision | 评估检索到的上下文`context`中，和参考答案`ground truth`一致的结果是否靠前 | LLM | 是 |
| 检索 | 上下文相关性 Context Relevancy | 评估检索到的上下文`context`中，和问题`question`的相关程度 | LLM | 否 |
| 生成 | 回答相关性 Answer Relevancy | 问题`question` 和答案 `answer`的相关性 | LLM | 否 |
| 生成 | 回答语义相似性 Answer semantic similarity | 答案 `answer`和参考答案`ground truth`的语义相似度 | 交叉编码器 | 是 |
| 生成 | 回答正确性 Answer Correctness | 答案 `answer`和参考答案`ground truth` 在事实方面的一致性 | LLM | 是 |
| 生成 | 回答忠诚度 Answer Faithfulness | 答案 `answer` 和上下文 `context`的事实一致性 | LLM | 否 |
| 生成 | 回答批评 Answer Critique | 对答案 `answer` 在指定的 Prompt 进行批评以识别有害内容 | LLM | 否 |

当数据集中没有人工标注的参考答案时，就需要使用那些不需要参考答案的指标，例如上下文相关性、回答相关性、回答忠诚度、回答批评等。

如果觉得评估指标过多，在有参考答案时，建议选择如下两个指标：

1. 上下文召回：用来评估检索效果。注意这里如果效果下降，可能原因是检索本身的效果不好，也可能是数据源缺少相关的数据。
2. 回答正确性：用来评估生成效果。

注：开源框架 ragas 中的 Prompt 和交叉编码器都是英文，不适合中文环境。

## 英文数据集，使用 ragas 评估


In [ ]:
%pip install datasets

In [ ]:
# 注意我们并没有实现 Provider 运行部分，实际的场景，Provider 可以为一个 Python脚本，用来调用你的 RAG 应用。

import json
from datasets import load_dataset

fiqa_eval = load_dataset("explodinggradients/fiqa", "ragas_eval")
with open("fiqa_eval.jsonl", 'w') as f:
    for data in fiqa_eval['baseline']:
        print(data)
        f.write(json.dumps(data, ensure_ascii=False) + '\n')

## 中文数据集，使用 ragas-zh 评估

TODO